In [1]:
import csv
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
from datetime import datetime

In [2]:
tweets = pd.read_csv('data/train.csv')
eval_ds = pd.read_csv('data/evaluation.csv')

Cleaning the datas

In [3]:
X = tweets.copy().drop('retweet_count', axis=1)
y = tweets.retweet_count.copy()

def day(t):
    dt = datetime.fromtimestamp(t / 1000)
    return dt.day

def clean(df):
    X = df.copy()
    X.timestamp = X.timestamp.apply(day)
    X.user_verified = X.user_verified.fillna(False).astype(int)
    X.user_statuses_count = X.user_statuses_count.fillna(0)
    X.user_followers_count = X.user_followers_count.fillna(0)
    X.user_friends_count = X.user_friends_count.fillna(0)
    X.user_mentions = X.user_mentions.fillna('').apply(lambda x: int(bool(x)))
    X.hashtags = X.hashtags.fillna('').astype(bool).astype(int)
    X.text = X.text.fillna('')
    X.urls = X.urls.fillna('')
    X.urls = X.urls.apply(lambda x: int(bool(x))) + X.text.apply(lambda x: int(bool('http' in x)))
    X.urls = X.urls.apply(lambda x: 1 if x==2 else x)
    return X

X = clean(X)

Insert a column for categories of words that get a high number of retweets

In [4]:
def insert_or(words, title, df):
    if title not in df.columns:
        df.insert(0, title, df.text.apply(lambda x: int(bool([1 for w in words if w in x.lower()]))))
        
def insert_cat(df):
    X = df.copy()
    trump = ['president', 'trump', 'donald', 'dt']
    insert_or(trump, 'trump', X)
    virus = ['virus', 'corona', 'covid', 'sick', 'flu']
    insert_or(virus, 'virus', X)
    doctor = ['doctor', 'nurse', 'kid', 'hospital', 'administration', 'medical', 'save']
    insert_or(doctor, 'doctor', X)
    america = ['america', 'democrat']
    insert_or(america, 'america', X)
    china = ['chin', 'wuhan']
    insert_or(china, 'china', X)
    handwash = ['hand', 'wash']
    insert_or(handwash, 'handwash', X)
    return X

X = insert_cat(X)

if 'text' in X.columns:
    X.insert(0, 'words_count', X.text.apply(lambda x: min(140, len(x.split(' ')))))
    X.drop('text', axis=1, inplace=True)

In [5]:
X.head(1)

,words_count,handwash,china,america,doctor,virus,trump,id,timestamp,user_verified,user_statuses_count,user_followers_count,user_friends_count,user_mentions,urls,hashtags
0,4,0,0,0,0,0,0,0,5,0,68460,1101,1226,0,0,0


In [15]:
y[X.timestamp == 2].describe()

count    77384.000000
mean         7.314458
std        320.201356
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max      46675.000000
Name: retweet_count, dtype: float64

In [ ]:
%%time
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

clf = make_pipeline(StandardScaler(), LinearSVC(penalty='l2', class_weight='balanced', random_state=0))
clf.fit(X, y)
scores_clf = cross_val_score(clf, X, y, cv=7)
print(scores_clf)

In [ ]:
from sklearn.ensemble import BaggingClassifier

In [ ]:
from sklearn.ensemble import DecisionTreeClassifier
seed = 8
base_cls = DecisionTreeClassifier() 

# no. of base classifier 
num_trees = 500

# bagging classifier 
model = BaggingClassifier(base_estimator = base_cls, 
                          n_estimators = num_trees, 
                          random_state = seed)


In [ ]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier()
rfc.fit(X, y)
scores = cross_val_score(rfc, X, y, cv=7)
print(scores)